In [47]:
# Python Program to Pull Historical Crypto Price Data from FTX API
# All FTX Markets shown here: https://ftx.com/markets
# 4 Coins Used for Program: BTC, ETH, SOL, XRP

In [48]:
# Required libaries to run FTX API program

import pandas as pd
import os
import json
import sys
import time
import hmac
import requests
import hvplot.pandas
from ftx_functions import (get_historical_data, get_summary_stats, get_average_spread)

In [49]:
# Pulls HISTORICAL SPOT prices for BTC, ETH, SOL, XRP
# Utilizes the function "get_historial_data" that 
# Takes one (1) parameter ("futures_ticker") as a (str)

btc_spot_historical = get_historical_data("BTC/USD")
eth_spot_historical = get_historical_data("ETH/USD")
sol_spot_historical = get_historical_data("SOL/USD")
xrp_spot_historical = get_historical_data("XRP/USD")


BTC/USD: Pulling Historical Price Data...
ETH/USD: Pulling Historical Price Data...
SOL/USD: Pulling Historical Price Data...
XRP/USD: Pulling Historical Price Data...


In [50]:
# Combines all the HISTORICAL SPOT price data into a single DataFrame

historical_spot_prices = pd.concat(
    [btc_spot_historical, 
     eth_spot_historical, 
     sol_spot_historical, 
     xrp_spot_historical], 
    axis="columns", join="inner")
display(historical_spot_prices)


,BTC/USD,ETH/USD,SOL/USD,XRP/USD
Date,,,,
2021-12-15 10:00:00,48266.0,3862.5,166.0700,0.804600
2021-12-15 11:00:00,48144.0,3853.1,165.8225,0.805975
2021-12-15 12:00:00,47607.0,3811.2,162.4525,0.797800
2021-12-15 13:00:00,47659.0,3820.4,163.9575,0.800350
2021-12-15 14:00:00,47379.0,3777.9,160.8450,0.795550
...,...,...,...,...
2022-02-15 17:00:00,43952.0,3099.3,101.4525,0.821450
2022-02-15 18:00:00,44155.0,3119.8,102.3650,0.825075
2022-02-15 19:00:00,44017.0,3110.9,101.6500,0.825825


In [51]:
# Pulls HISTORICAL FUTURES price data for BTC, ETH, SOL, XRP, MATIC
# Utilizes the function "get_historial_data" that 
# Takes one (1) parameter ("futures_ticker") as a (str)

btc_futures_historical = get_historical_data("BTC-20211231")
eth_futures_historical = get_historical_data("ETH-20211231")
sol_futures_historical = get_historical_data("SOL-20211231")
xrp_futures_historical = get_historical_data("XRP-20211231")


BTC-20211231: Pulling Historical Price Data...
ETH-20211231: Pulling Historical Price Data...
SOL-20211231: Pulling Historical Price Data...
XRP-20211231: Pulling Historical Price Data...


In [52]:
# Combines all the HISTORICAL FUTURES price data into a single DataFrame

historical_futures_prices = pd.concat(
    [btc_futures_historical, 
     eth_futures_historical, 
     sol_futures_historical, 
     xrp_futures_historical], 
    axis="columns", join="inner")

display(historical_futures_prices)


,BTC-20211231,ETH-20211231,SOL-20211231,XRP-20211231
Date,,,,
2021-12-15 10:00:00,48384.0,3867.0,166.1900,0.80405
2021-12-15 11:00:00,48269.0,3857.8,166.0600,0.80600
2021-12-15 12:00:00,47711.0,3813.6,162.7000,0.79760
2021-12-15 13:00:00,47747.0,3822.8,164.1300,0.80025
2021-12-15 14:00:00,47457.0,3781.3,160.9125,0.79480
...,...,...,...,...
2021-12-30 23:00:00,47106.0,3704.1,172.4325,0.83985
2021-12-31 00:00:00,47090.0,3695.9,172.7300,0.83570
2021-12-31 01:00:00,47163.0,3721.4,173.1225,0.83795


In [53]:
# Bitcoin Futures & Spot Price DataFrame
btc_prices = pd.concat([btc_futures_historical, btc_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in BTC, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
btc_prices["Arbitrage Spread"] = btc_prices["BTC-20211231"] - btc_prices["BTC/USD"]
btc_prices["Arbitrage Spread as % of Spot Price"] = btc_prices["Arbitrage Spread"] / btc_prices["BTC/USD"]
btc_prices["30-Day SMA of Spread in USD"] = btc_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()

btc_prices


,BTC-20211231,BTC/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in USD
Date,,,,,
2021-12-15 10:00:00,48384.0,48266.0,118.0,0.002445,NaN
2021-12-15 11:00:00,48269.0,48144.0,125.0,0.002596,NaN
2021-12-15 12:00:00,47711.0,47607.0,104.0,0.002185,NaN
2021-12-15 13:00:00,47747.0,47659.0,88.0,0.001846,NaN
2021-12-15 14:00:00,47457.0,47379.0,78.0,0.001646,NaN
...,...,...,...,...,...
2021-12-30 23:00:00,47106.0,47126.0,-20.0,-0.000424,10.100000
2021-12-31 00:00:00,47090.0,47079.0,11.0,0.000234,9.866667
2021-12-31 01:00:00,47163.0,47131.0,32.0,0.000679,11.166667


In [54]:
# Bitcoin Futures vs Spot Price Chart

btc_prices_plot = btc_prices.hvplot.line(
    x="Date", y= ["BTC-20211231", "BTC/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Bitcoin Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Bitcoin Futures & Spot Markets")
btc_prices_plot


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [55]:
# Bitcoin Arbitrage Spread as & of Spot (Visualization)

btc_arbitrage_plot = btc_prices.hvplot.line(
    x="Date", y= "Arbitrage Spread as % of Spot Price",
    height=400, width=1000,
    xlabel='Date', ylabel = "Arbitrage Spread as % of Spot Price ",
    legend='top_left',
    title = "Bitcoin Arbitrage Spread")
btc_arbitrage_plot


:Curve   [Date]   (Arbitrage Spread as % of Spot Price)

In [56]:
# Bitcoin 30-Day Simple Moving Average Arbitrage Spread (Visualization)

btc_30d_arbitrage_plot = btc_prices.hvplot.line(
    x="Date", y= "30-Day SMA of Spread in USD",
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA Arbitrage Spread in USD",
    legend='top_left',
    title = "Bitcoin Arbitrage Spread: 30 Day Simple Moving Average")

btc_30d_arbitrage_plot

:Curve   [Date]   (30-Day SMA of Spread in USD)

In [57]:
# Ethereum Futures & Spot Price DataFrame
eth_prices = pd.concat([eth_futures_historical, eth_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in ETH, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
eth_prices["Arbitrage Spread"] = eth_prices["ETH-20211231"] - eth_prices["ETH/USD"]
eth_prices["Arbitrage Spread as % of Spot Price"] = eth_prices["Arbitrage Spread"] / eth_prices["ETH/USD"]
eth_prices["30-Day SMA of Spread in USD"] = eth_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()
eth_prices


,ETH-20211231,ETH/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in USD
Date,,,,,
2021-12-15 10:00:00,3867.0,3862.5,4.5,0.001165,NaN
2021-12-15 11:00:00,3857.8,3853.1,4.7,0.001220,NaN
2021-12-15 12:00:00,3813.6,3811.2,2.4,0.000630,NaN
2021-12-15 13:00:00,3822.8,3820.4,2.4,0.000628,NaN
2021-12-15 14:00:00,3781.3,3777.9,3.4,0.000900,NaN
...,...,...,...,...,...
2021-12-30 23:00:00,3704.1,3709.5,-5.4,-0.001456,-3.080000
2021-12-31 00:00:00,3695.9,3701.7,-5.8,-0.001567,-3.270000
2021-12-31 01:00:00,3721.4,3726.7,-5.3,-0.001422,-3.410000


In [58]:
# Ethereum Futures vs Spot Price Chart

eth_prices_plot = eth_prices.hvplot.line(
    x="Date", y= ["ETH-20211231", "ETH/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Ethereum Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Ethereum Futures & Spot Markets")
eth_prices_plot


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [59]:
# Ethereum Arbitrage Spread Chart

eth_arbitrage_plot = eth_prices.hvplot.line(
    x="Date", y= "Arbitrage Spread as % of Spot Price",
    height=400, width=1000,
    xlabel='Date', ylabel = "Aritrage Spread as % of Spot Price",
    legend='top_left',
    title = "Ethereum Arbitrage Spread")

eth_arbitrage_plot

:Curve   [Date]   (Arbitrage Spread as % of Spot Price)

In [60]:
# Ethereum 30-Day Simple Moving Average Arbitrage Spread Chart

eth_30d_arbitrage_plot = eth_prices.hvplot.line(
    x="Date", y= "30-Day SMA of Spread in USD",
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA of Spread in USD",
    legend='top_left',
    title = "Ethereum Arbitrage Spread: 30 Day Simple Moving Average")

eth_30d_arbitrage_plot


:Curve   [Date]   (30-Day SMA of Spread in USD)

In [61]:
# Solana Futures & Spot Price DataFrame concatenation 
sol_prices = pd.concat([sol_futures_historical, sol_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in SOL, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
sol_prices["Arbitrage Spread"] = sol_prices["SOL-20211231"] - sol_prices["SOL/USD"]
sol_prices["Arbitrage Spread as % of Spot Price"] = sol_prices["Arbitrage Spread"] / sol_prices["SOL/USD"]
sol_prices["30-Day SMA of Spread in SOL"] = sol_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()

sol_prices


,SOL-20211231,SOL/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in SOL
Date,,,,,
2021-12-15 10:00:00,166.1900,166.0700,0.1200,0.000723,NaN
2021-12-15 11:00:00,166.0600,165.8225,0.2375,0.001432,NaN
2021-12-15 12:00:00,162.7000,162.4525,0.2475,0.001524,NaN
2021-12-15 13:00:00,164.1300,163.9575,0.1725,0.001052,NaN
2021-12-15 14:00:00,160.9125,160.8450,0.0675,0.000420,NaN
...,...,...,...,...,...
2021-12-30 23:00:00,172.4325,172.5325,-0.1000,-0.000580,-0.054500
2021-12-31 00:00:00,172.7300,172.6700,0.0600,0.000347,-0.046750
2021-12-31 01:00:00,173.1225,173.0250,0.0975,0.000564,-0.039667


In [62]:
# Solana Futures vs Spot Price Chart

sol_prices_plot = sol_prices.hvplot.line(
    x="Date", y= ["SOL-20211231", "SOL/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Solana Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Solana Futures & Spot Markets")

sol_prices_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [63]:
# Solana Arbitrage Spread Chart

sol_arbitrage_plot = sol_prices.hvplot.line(
    x="Date", y= ["Arbitrage Spread as % of Spot Price"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Arbitrage Spread as % of Spot Price",
    legend='top_left',
    title = "Solana Arbitrage Spread")

sol_arbitrage_plot

:Curve   [Date]   (Arbitrage Spread as % of Spot Price)

In [64]:
# Solana 30-Day Simple Moving Average Arbitrage Spread Chart

sol_30d_arbitrage_plot = sol_prices.hvplot.line(
    x="Date", y= "30-Day SMA of Spread in SOL",
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA of Spread in USD",
    legend='top_left',
    title = "Solana Arbitrage Spread: 30-Day Simple Moving Average")

sol_30d_arbitrage_plot


:Curve   [Date]   (30-Day SMA of Spread in SOL)

In [65]:
# Ripple Futures & Spot Price DataFrame concatenation
xrp_prices = pd.concat([xrp_futures_historical, xrp_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in XRP, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
xrp_prices["Arbitrage Spread"] = xrp_prices["XRP-20211231"] - xrp_prices["XRP/USD"]
xrp_prices["Arbitrage Spread as % of Spot Price"] = xrp_prices["Arbitrage Spread"] / xrp_prices["XRP/USD"]
xrp_prices["30-Day SMA of Spread in XRP"] = xrp_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()

xrp_prices


,XRP-20211231,XRP/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in XRP
Date,,,,,
2021-12-15 10:00:00,0.80405,0.804600,-0.000550,-0.000684,NaN
2021-12-15 11:00:00,0.80600,0.805975,0.000025,0.000031,NaN
2021-12-15 12:00:00,0.79760,0.797800,-0.000200,-0.000251,NaN
2021-12-15 13:00:00,0.80025,0.800350,-0.000100,-0.000125,NaN
2021-12-15 14:00:00,0.79480,0.795550,-0.000750,-0.000943,NaN
...,...,...,...,...,...
2021-12-30 23:00:00,0.83985,0.838500,0.001350,0.001610,0.000653
2021-12-31 00:00:00,0.83570,0.835300,0.000400,0.000479,0.000621
2021-12-31 01:00:00,0.83795,0.836375,0.001575,0.001883,0.000652


In [66]:
# Ripple Futures vs Spot Price Chart

xrp_prices_plot = xrp_prices.hvplot.line(
    x="Date", y= ["XRP-20211231", "XRP/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Ripple Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Ripple Futures & Spot Markets")

xrp_prices_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [67]:
# Ripple Arbitrage Spread Chart

xrp_arbitrage_plot = xrp_prices.hvplot.line(
    x="Date", y= "Arbitrage Spread as % of Spot Price",
    height=400, width=1000,
    xlabel='Date', ylabel = "Arbitrage Spread as % of Spot Price",
    legend='top_left',
    title = "Ripple Arbitrage Spread")

xrp_arbitrage_plot


:Curve   [Date]   (Arbitrage Spread as % of Spot Price)

In [68]:
# Ripple 30-Day Simple Moving Average Arbitrage Spread Chart

xrp_30d_arbitrage_plot = xrp_prices.hvplot.line(
    x="Date", y= "30-Day SMA of Spread in XRP",
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA of Spread in USD",
    legend='top_left',
    title = "Ripple Arbitrage Spread: 30-Day Simple Moving Average")

xrp_30d_arbitrage_plot


:Curve   [Date]   (30-Day SMA of Spread in XRP)

In [69]:
# Function to "get_summary_stats" for each asset 
# Function takes three (3) parameters (asset_dataframe, spot_ticker, asset_name)

btc_summary = get_summary_stats(btc_prices, "BTC/USD", "Bitcoin")
eth_summary = get_summary_stats(eth_prices, "ETH/USD", "Ethereum")
sol_summary = get_summary_stats(sol_prices, "SOL/USD", "Solana")
xrp_summary = get_summary_stats(xrp_prices, "XRP/USD", "Ripple")


In [70]:
# Summary statistics for all asset combine into a single DataFrame
arbitrage_summary_statistics = pd.concat([btc_summary, eth_summary, sol_summary, xrp_summary], axis="columns", join="inner")

# Chart shows arbitrage spreads as a percentrage of the asset's spot price for comparison
arbitrage_summary_statistics

,Bitcoin,Ethereum,Solana,Ripple
count,378.000000,378.000000,378.000000,378.000000
mean,0.001470,0.000380,0.000233,-0.002335
std,0.000837,0.000756,0.000701,0.004298
min,-0.004830,-0.007667,-0.001713,-0.014491
25%,0.001065,0.000049,-0.000223,-0.001944
50%,0.001544,0.000474,0.000209,-0.000517
75%,0.002069,0.000821,0.000705,0.000353
max,0.003002,0.001891,0.002312,0.002614


In [71]:
# Function to "get_average_spread" for each asset over entire period

btc_average_spread = get_average_spread(btc_prices, "BTC/USD", "Bitcoin")
eth_average_spread = get_average_spread(eth_prices, "ETH/USD", "Ethereum")
sol_average_spread = get_average_spread(sol_prices, "SOL/USD", "Solana")
xrp_average_spread = get_average_spread(xrp_prices, "XRP/USD", "Ripple")


# Combines data into a single DataFrame
average_spreads = pd.concat(
    [btc_average_spread, 
     eth_average_spread, 
     sol_average_spread, 
     xrp_average_spread], 
    axis="rows", join="inner")

# Bar Chart of Historical Arbitrage Spreads per Day on Average
spreads_bar_chart = average_spreads.hvplot.bar(
    title='Average Daily Arbitrage Spreads as % of Spot Price'
    ).opts(
    yformatter='%.5f',
    hover_color='yellow')

display(spreads_bar_chart)

:Bars   [index]   (Historical Daily Arbitrage Spreads)